# Visualization of CARTaGENE population structure

To run this notebook specifically as in the manuscript:
* load ipython/3.8
* load scipy-stack/2020b
* shut down the kernel
* select the new kernel (should be python 3.8.*)

These are all relative to my directories. Where you see `...`, insert your own paths.

For the entire pipeline, you will need to do the following:

* Generate the principal components after having filtered for LD and HLA. You can use PLINK and/or flashPCA (the latter outputs an estimated variance explained).
* Generate 2D or 3D UMAP embeddings (for visualization); 2D embeddings are good for static images, 3D are good for interactive exploration with `plotly`. These generally have `n_neighbors` set between 10 and 25. Setting this very low (e.g. 3-5) can result in small clusters of related individuals. `min_dist` is set between 0.3 and 0.5 to allow for better visualizations. I vary the number of input PCs to see if there are any interesting differences or patterns with more or less data. 
* Generate UMAP embeddings in 3 or higher dimensions for density-based clustering using HDBSCAN. These UMAP embeddings should have `min_dist` set very low (equal or close to 0).
* Generate clusters using HDBSCAN$(\hat{\epsilon})$. Setting the $\hat{\epsilon}$ parameter (split-merger threshold) at 0.5 works reasonably well for biobank data. These parameter combinations tend to highlight both very local and global levels of population structure (e.g. structure within Quebec as well as continental/sub-continental population structure).
* Colour the 2D UMAP embeddings using the HDBSCAN clusters.

The code here focuses on generating visualizations. My strategy has been to generate many embeddings and clusterings using a grid search. For visualization I find using 10-25 PCs is useful. For which clustering to use, I usually choose one that has (1) many/the most clusters; and (2) every individual or almost every individual belongs to a cluster. I recommend using a range of PCs (there is pretty much always signal in higher-order PCs; see the paper on topological stratificaton for details). For the plots, I recommend removing tick-marks from axes as UMAP distances are not interpretable. I also usually plot the largest clusters first to make sure I don't smother smaller groups.

I put together the final figures using external software but you could probably do it if you're clever enough with some combination of matplotlib, ggplot2, and imagemagick.

For a fuller exploration of UMAP/HDBSCAN for population genetics, see the following manuscripts:
* [Diaz-Papkovich, Alex, et al. "UMAP reveals cryptic population structure and phenotype heterogeneity in large genomic cohorts." PLoS genetics 15.11 (2019): e1008432.](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1008432)
* [Diaz-Papkovich, Alex, et al. "Topological stratification of continuous genetic variation in large biobanks." bioRxiv (2023): 2023-07.](https://www.biorxiv.org/content/10.1101/2023.07.06.548007.abstract)

For more general scripts for UMAP and clustering see https://github.com/diazale/topstrat

## Installations

Need to install word cloud and font packages. Once installed, restart the kernel. It might take a few attemps to install these.

In [ ]:
#!pip install wordcloud
#!pip install fonttools

## Import data

In [ ]:
%matplotlib inline
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image # For displaying documentation images

In [ ]:
subdir = "/hla_removed_ld_thinned_0.1" # Sub-directories for specific runs. I used a run with LD thinning and HLA removed.

# Key directories and files
# You will also have to import various files of questionnaire data, however these directories have already changed since I wrote this code
cluster_dir = ".../cartagene/hdbscan_clusters" + subdir # Cluster locations
viz_dir = ".../cartagene/projections_viz" + subdir # 2D and 3D UMAP plots for visualization

In [ ]:
# The file names generated by my UMAP and HDBSCAN scripts contain their parameters.
# This code extracts the parameters in the file names.
def extract_parameters(fname_):
    # Input: Clustering file name
    # Output: Parameters used to generate the clustering
    
    # Variables:
    # Minimum points
    # Epsilon
    # PCs
    # NC
    # NN
    # MD
    mp_ = int(fname_.split("_min")[1].split("_")[0])
    eps_ = float(fname_.split("_EPS")[1].split("_")[0])
    pcs_ = int(fname_.split("_PC")[1].split("_")[0])
    nc_ = int(fname_.split("_NC")[1].split("_")[0])
    nn_ = int(fname_.split("_NN")[1].split("_")[0])
    md_ = float(fname_.split("_MD")[1].split("_")[0])
    
    return mp_, eps_, pcs_, nc_, nn_, md_

In [ ]:
# Import the clusterings and create a data frame to store their details and parameterizations
# Using this we can look at all of the clusterings and see (1) how many clusters there are; and (2) how many individuals are not in a cluster
cluster_properties = list()

for c in os.listdir(cluster_dir):
    if os.path.isfile(os.path.join(cluster_dir, c)):
        cl = np.loadtxt(os.path.join(cluster_dir,c))
        num_unclustered = np.sum(cl==-1)
        num_clusters = len(set(cl))
        mp, e, p, nc, nn, md = extract_parameters(c)
        cluster_properties.append([mp,e,p,nc,nn,md,num_unclustered,num_clusters,c])
        
cluster_properties_df = pd.DataFrame(cluster_properties,
                                     columns=["min_points","eps","pcs","nc","nn","md","unclustered","clusters","name"])

# Write to disk
cluster_properties_df.to_csv(out_dir + "/cluster_properties.csv", index=False)

In [ ]:
# These are the variables I used to generate word clouds
# I selected these to try and understand the demographic histories underlying the population structure in our genetic data
# They are the collection region, country of birth, province of birth (if born in Canada), and selected ethnicity variables
# For brevity I have excluded a series of splits and mergers that are necessary to generate the full data set (geno_df_analysis)
# This should be straightforward (if somewhat tedious) to generate using various split/merge commands
variables_of_interest = [
"Cregion",
"PROVINCE_BIRTH",
"COUNTRY_BIRTH",
"COUNTRY_BIRTH_OTHER",
"ETHNICITY_ME",
"ethnic_fr",
"ETHNIC_FR_CB",
"ethnicity"]

In [ ]:
geno_df_analysis = ... # Take some steps here to connect questionnaire data to dimensionally-reduced genotype data (i.e. UMAP and PCA coordinates)

In [ ]:
# Import clusters
clustering = "hdbscan_labels_min25_EPS0.3_hla_removed_ld_thinned.eigenvec_UMAP_PC25_NC3_NN10_MD0.001_euclidean_20220714_173646.txt"
clusters = np.loadtxt(os.path.join(cluster_dir,clustering))

# Copy data frame and attach clusters
geno_df_analysis = geno_aux_data.copy()
geno_df_analysis["cluster"] = clusters

## Plotting PCA and UMAP with clusters

In [ ]:
# Colour palette
pal = [
    "#238BC1",
    "#203800",
    "#32AC39",
    "#E03D34",
    "#A67FC9",
    "#9F6A5E",
    "#EB8FCD",
    "#919191",
    "#C8C62C",
    "#610A24",
    "#9c6146",
    "#007352",
    "#A22188",
    "#FF9209",
    "#00C8D8"]

In [ ]:
# Import the embedding for visualization and the array of cluster labels to use
viz_file = "hla_removed_ld_thinned.eigenvec_UMAP_PC10_NC2_NN15_MD0.5_euclidean_20220714_174506.txt" # original UMAP used for viz

temp_proj = np.loadtxt(os.path.join(viz_dir, viz_file))
cluster_array = geno_df_analysis.cluster.values

In [ ]:
# Plot PCA + clusters
temp_proj = pca
cluster_array = geno_df_analysis.cluster.values

plt.figure(figsize=(10, 10))
for cl in list(set(list(clusters))):
    print(cl)
    plt.plot(temp_proj[np.where(cluster_array==cl),0], 
             temp_proj[np.where(cluster_array==cl),1], 
                       '.', color = pal[int(cl)],
            markersize=2.5, alpha=0.4)

plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.rcParams.update({"font.size":22})

fig_path = "..."
plt.savefig(os.path.join(fig_path,"cartagene_flagship_clusters_pca.png"),dpi=300,
           bbox_inches="tight")

In [ ]:
# Plot UMAP + clusters
plt.figure(figsize=(10, 10))

for cl in geno_df_analysis["cluster"].value_counts().index.values.tolist():
    print(cl)
    plt.plot(temp_proj[np.where(cluster_array==cl),0], 
             temp_proj[np.where(cluster_array==cl),1], 
                       '.', color = pal[int(cl)],
            markersize=2.5, alpha=0.6)

plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.rcParams.update({"font.size":22})

fig_path = "..."
plt.savefig(os.path.join(fig_path,"cartagene_flagship_clusters.png"),dpi=300,
           bbox_inches="tight")

## Generate word clouds

In [ ]:
# Generates word clouds for specified variables for each cluster.
# Colours will match clusters
fig_path = "..."
cob_var = "COUNTRY_BIRTH"

# Get the COBs from the cluster
for cl in list(set(clusters)):    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    
    cluster_cobs = geno_df_analysis[geno_df_analysis["cluster"]==cl][cob_var].values.tolist()
    cluster_cobs = [c for c in cluster_cobs if str(c)!= "nan"]
    
    unique_str = (" ").join(cluster_cobs)
    wc = WordCloud(font_path="/usr/share/fonts/dejavu/DejaVuSansCondensed.ttf",
                   width=2000,
                   height=2000,
                   relative_scaling=0.8,
                   prefer_horizontal=1,
                   mode="RGBA",
                   background_color=pal[int(cl)]+"11",
                   collocations=False,
                   color_func=lambda *args, **kwargs: pal[int(cl)]).generate(unique_str)

    plt.xticks([], [])
    plt.yticks([], [])

    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_color(pal[int(cl)])
        ax.spines[axis].set_linewidth(3)

    plt.imshow(wc)
    
    plt.savefig(os.path.join(fig_path,"cartagene_wordclouds_" + cob_var + str(int(cl)) + ".png"),dpi=300,bbox_inches="tight")